In [27]:
import numpy as np
import random as rd
from playground import monopoly_with_fair_prison

rd.seed(42)
NUM_SIMULATIONS = 200
# 固定數值
GAME_MAP = [rd.randint(400,500) for i in range(10)]
PLAYER_NUM = 4
PRISON_LOCATION = 5
PRISON_DAY = 2

# 輸入變動數值
# PLAYERS_MONEY
# PRISON_FINE

In [ ]:
input_data = np.array([[rd.randint(500,2000),rd.randint(35,150)] for i in range(3000)])
input_data

array([[2733,   46],
       [2918,   89],
       [ 630,   38],
       ...,
       [2362,   86],
       [2854,   35],
       [1569,   57]], shape=(3000, 2))

In [29]:
output = []
for i in input_data:
    turn = []
    PLAYERS_MONEY = i[0]
    PRISON_FINE = i[1]
    for j in range(NUM_SIMULATIONS):
        game = monopoly_with_fair_prison(PLAYERS_MONEY,PLAYER_NUM,GAME_MAP,PRISON_LOCATION,PRISON_DAY,j,PRISON_FINE)
        game.play_game()
        turn.append(game.get_turn())
    output.append(round(np.mean(turn),2))
output


[np.float64(173.94),
 np.float64(140.58),
 np.float64(26.41),
 np.float64(22.44),
 np.float64(23.64),
 np.float64(215.72),
 np.float64(160.38),
 np.float64(125.12),
 np.float64(22.78),
 np.float64(164.24),
 np.float64(17.73),
 np.float64(18.74),
 np.float64(87.68),
 np.float64(42.48),
 np.float64(21.12),
 np.float64(66.2),
 np.float64(20.67),
 np.float64(20.59),
 np.float64(47.28),
 np.float64(26.97),
 np.float64(17.1),
 np.float64(86.77),
 np.float64(18.87),
 np.float64(19.97),
 np.float64(32.06),
 np.float64(55.17),
 np.float64(20.52),
 np.float64(25.16),
 np.float64(22.3),
 np.float64(49.38),
 np.float64(21.74),
 np.float64(19.57),
 np.float64(34.88),
 np.float64(80.18),
 np.float64(67.58),
 np.float64(20.99),
 np.float64(29.16),
 np.float64(20.2),
 np.float64(20.3),
 np.float64(29.92),
 np.float64(97.22),
 np.float64(30.09),
 np.float64(155.22),
 np.float64(37.4),
 np.float64(23.65),
 np.float64(15.98),
 np.float64(46.69),
 np.float64(39.45),
 np.float64(21.59),
 np.float64(53.18),

In [30]:
output = np.array(output)

In [31]:
output

array([173.94, 140.58,  26.41, ...,  96.86, 225.3 ,  35.52], shape=(3000,))

In [80]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
input_data_norm = normalize(input_data,norm='l1')
X_train, X_test, y_train, y_test = train_test_split(input_data_norm,output , 
                                   random_state=104,  
                                   test_size=0.3,  
                                   shuffle=True) 

In [81]:
X_train

array([[0.9630494 , 0.0369506 ],
       [0.93765903, 0.06234097],
       [0.96117216, 0.03882784],
       ...,
       [0.96232102, 0.03767898],
       [0.92039424, 0.07960576],
       [0.81763827, 0.18236173]], shape=(2100, 2))

In [ ]:
model = MLPRegressor(solver="adam",activation="relu",hidden_layer_sizes=[2,10,5,1])

In [83]:
model.fit(X_train,y_train)

c:\Users\User\OneDrive - mail.nchu.edu.tw\碩零\暑假作業\大富翁報告\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressor(hidden_layer_sizes=[2, 10, 5, 1])

In [78]:
y_pred = model.predict(X_train)

In [79]:
from sklearn.metrics import root_mean_squared_error
root_mean_squared_error(y_train,y_pred)

77.14313755904423